# Stockholm Archipelago Trail POI Map
This notebook fetches Stockholm Archipelago Trail sections from Wikidata and displays drinking water, toilets, restaurants, hotels, campsites and hostels within 500 m of the trail on a Folium map. 
* [Issue 153](https://github.com/salgo60/Stockholm_Archipelago_Trail/pull/153)
* This [notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/sat_poi_folium_map.ipynb)

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))




Start: 2025-09-07 19:26:12


In [2]:
import pandas as pd
import requests
import folium


In [ ]:
def fetch_sat_sections_from_wikidata(trail_qid: str = "Q131318799") -> pd.DataFrame:
    query = f"""
    SELECT ?section ?sectionLabel ?osmr ?osmw_p10689 ?osmw_p11693 WHERE {{
      VALUES ?trail {{ wd:{trail_qid} }}
      {{
        ?section wdt:P361+ ?trail .
      }} UNION {{
        ?trail wdt:P527 ?section .
      }}
      FILTER(?section != ?trail)
      OPTIONAL {{ ?section wdt:P402 ?osmr }}             # OSM relation id
      OPTIONAL {{ ?section wdt:P10689 ?osmw_p10689 }}    # OSM way id (alt 1)
      OPTIONAL {{ ?section wdt:P11693 ?osmw_p11693 }}    # OSM way id (alt 2)
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "sv,en". }}
    }}
    ORDER BY ?sectionLabel
    """
    url = "https://query.wikidata.org/sparql"
    headers = {"Accept": "application/sparql-results+json"}
    response = requests.get(url, params={"query": query}, headers=headers)
    response.raise_for_status()
    data = response.json()
    rows = []
    for item in data['results']['bindings']:
        rows.append({
            'section': item['section']['value'],
            'sectionLabel': item.get('sectionLabel', {}).get('value'),
            'osmr': item.get('osmr', {}).get('value'),
            'osmw_p10689': item.get('osmw_p10689', {}).get('value'),
            'osmw_p11693': item.get('osmw_p11693', {}).get('value'),
        })
    df = pd.DataFrame(rows)
    for col in ['osmr','osmw_p10689','osmw_p11693']:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
    return df


In [ ]:
sections_df = fetch_sat_sections_from_wikidata()
sections_df.head()


In [ ]:
def fetch_pois_for_section(osm_id: int, id_type: str, dist: int = 500):
    if id_type == 'relation':
        pivot = f'way(r:{osm_id})'
    else:
        pivot = f'way({osm_id})'
    query = f"""
    [out:json][timeout:25];
    {pivot}->.trail;
    (
      nwr(around:{dist}, .trail)["amenity"="drinking_water"];
      nwr(around:{dist}, .trail)["amenity"="toilets"];
      nwr(around:{dist}, .trail)["amenity"="restaurant"];
      nwr(around:{dist}, .trail)["tourism"="hotel"];
      nwr(around:{dist}, .trail)["tourism"="camp_site"];
      nwr(around:{dist}, .trail)["tourism"="hostel"];
    );
    out center tags;
    """
    response = requests.get('https://overpass-api.de/api/interpreter', params={'data': query})
    response.raise_for_status()
    data = response.json()
    pois = []
    for el in data['elements']:
        if 'lat' in el and 'lon' in el:
            lat, lon = el['lat'], el['lon']
        elif 'center' in el:
            lat, lon = el['center']['lat'], el['center']['lon']
        else:
            continue
        tags = el.get('tags', {})
        tags.update({'osm_id': el['id'], 'osm_type': el['type']})
        pois.append({'lat': lat, 'lon': lon, **tags})
    return pois


In [ ]:
all_pois = []
for _, row in sections_df.iterrows():
    if pd.notnull(row['osmr']):
        all_pois.extend(fetch_pois_for_section(int(row['osmr']), 'relation'))
    elif pd.notnull(row['osmw_p10689']):
        all_pois.extend(fetch_pois_for_section(int(row['osmw_p10689']), 'way'))
    elif pd.notnull(row['osmw_p11693']):
        all_pois.extend(fetch_pois_for_section(int(row['osmw_p11693']), 'way'))
poi_df = pd.DataFrame(all_pois)
poi_df.head()


In [ ]:
m = folium.Map(location=[59.3, 18.0], zoom_start=8)
category_colors = {
    'drinking_water': 'blue',
    'toilets': 'green',
    'restaurant': 'red',
    'hotel': 'purple',
    'camp_site': 'orange',
    'hostel': 'gray',
}
for _, feature in poi_df.iterrows():
    lat, lon = feature['lat'], feature['lon']
    if feature.get('amenity') in category_colors:
        key = feature['amenity']
    elif feature.get('tourism') in category_colors:
        key = feature['tourism']
    else:
        continue
    folium.CircleMarker([lat, lon], radius=5, color=category_colors[key], fill=True, fill_color=category_colors[key], popup=key).add_to(m)
m


In [ ]:
m.save('../html/sat_153_poi_map.html')


In [ ]:
# End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))
